In [45]:
import pandas as pd
import numpy as np

In [46]:
df = pd.read_csv('Seoul_1.csv')
df

,자치구명,계약일,건물면적(㎡),층,건축년도,연도,물건금액(만원)
0,동대문구,20221231,84.97,11,2003.0,2022,76500
1,구로구,20221231,59.88,4,2011.0,2022,85000
2,노원구,20221231,41.30,12,1988.0,2022,39000
3,동대문구,20221231,59.98,12,2011.0,2022,80000
4,송파구,20221231,100.82,1,1988.0,2022,185000
...,...,...,...,...,...,...,...
560753,구로구,20160101,84.85,18,1998.0,2016,30500
560754,노원구,20160101,59.82,7,2005.0,2016,30800
560755,서초구,20160101,134.36,12,2005.0,2016,94000
560756,양천구,20160101,49.99,2,2000.0,2016,21000


### Kospi종가 지수 컬럼 붙이기

In [47]:
import yfinance as yf

# KOSPI 데이터 가져오기
kospi = yf.download('^KS11', start='2016-01-01', end='2022-12-31')
kospi

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-01-04,1954.469971,1954.520020,1918.760010,1918.760010,1918.760010,359000
2016-01-05,1911.930054,1937.569946,1911.930054,1930.530029,1930.530029,446500
2016-01-06,1934.250000,1934.250000,1911.609985,1925.430054,1925.430054,594600
2016-01-07,1915.709961,1926.410034,1901.239990,1904.329956,1904.329956,393000
2016-01-08,1889.420044,1918.250000,1883.819946,1917.619995,1917.619995,430200
...,...,...,...,...,...,...
2022-12-23,2325.860107,2333.080078,2311.899902,2313.689941,2313.689941,367000
2022-12-26,2312.540039,2321.919922,2304.199951,2317.139893,2317.139893,427600
2022-12-27,2327.520020,2335.989990,2321.479980,2332.790039,2332.790039,448300


In [48]:
# 종가만 남기고 Date컬럼 시계열인거 확인
kospi = kospi[['Close']]
kospi = kospi.reset_index()
kospi

,Date,Close
0,2016-01-04,1918.760010
1,2016-01-05,1930.530029
2,2016-01-06,1925.430054
3,2016-01-07,1904.329956
4,2016-01-08,1917.619995
...,...,...
1712,2022-12-23,2313.689941
1713,2022-12-26,2317.139893
1714,2022-12-27,2332.790039
1715,2022-12-28,2280.449951


In [49]:
# 기존 데이터프레임 시계열형태 변환 및 kospi지수 합치기
# 컬럼 형태 변환
df['계약일'] = df['계약일'].astype(int).astype(str)

# 시계열로 변환
df['계약일'] = pd.to_datetime(df['계약일'], format='%Y-%m-%d')

In [50]:
# kospi의 Date컬럼 merge를 위해 df의 계약년월로 이름 바꾸기
kospi = kospi.rename(columns={'Date' : '계약일'})

In [52]:
# 계약일을 기준으로 left조인
merged_df = pd.merge(df, kospi, on='계약일', how='left')

### Fedrate컬럼 합치기

In [55]:
fed = pd.read_csv('Fedrate.csv')

In [56]:
fed

,DATE,FEDFUNDS
0,2016-01-01,0.34
1,2016-02-01,0.38
2,2016-03-01,0.36
3,2016-04-01,0.37
4,2016-05-01,0.37
...,...,...
79,2022-08-01,2.33
80,2022-09-01,2.56
81,2022-10-01,3.08
82,2022-11-01,3.78


In [61]:
# 기존데이터프레임과 left 조인하기 위해 컬럼명 일치화
fed = fed.rename(columns={'DATE' : '계약일'})

In [62]:
# 1.fed데이터 시계열 변경
fed['계약일'] = pd.to_datetime(fed['계약일'], format='%Y-%m-%d')

In [71]:
# 2.fed 데이터는 월별로 나눠져있기에 기존 데이터프레임도 열별로 나뉜 컬럼 생성
merged_df['월'] = merged_df['계약일'].dt.to_period('M')
fed['월'] = fed['계약일'].dt.to_period('M')

In [72]:
fed

,계약일,FEDFUNDS,월
0,2016-01-01,0.34,2016-01
1,2016-02-01,0.38,2016-02
2,2016-03-01,0.36,2016-03
3,2016-04-01,0.37,2016-04
4,2016-05-01,0.37,2016-05
...,...,...,...
79,2022-08-01,2.33,2022-08
80,2022-09-01,2.56,2022-09
81,2022-10-01,3.08,2022-10
82,2022-11-01,3.78,2022-11


In [73]:
merged_df2 = pd.merge(merged_df, fed, on='월', how='left')

In [77]:
#중복되는 계약일_y삭제
merged_df2 = merged_df2.drop(columns = ['계약일_y'])

In [112]:
merged_df2

,자치구명,계약일_x,건물면적(㎡),층,건축년도,연도,물건금액(만원),Close,월,FEDFUNDS
0,동대문구,2022-12-31,84.97,11,2003.0,2022,76500,NaN,2022-12,4.10
1,구로구,2022-12-31,59.88,4,2011.0,2022,85000,NaN,2022-12,4.10
2,노원구,2022-12-31,41.30,12,1988.0,2022,39000,NaN,2022-12,4.10
3,동대문구,2022-12-31,59.98,12,2011.0,2022,80000,NaN,2022-12,4.10
4,송파구,2022-12-31,100.82,1,1988.0,2022,185000,NaN,2022-12,4.10
...,...,...,...,...,...,...,...,...,...,...
560753,구로구,2016-01-01,84.85,18,1998.0,2016,30500,NaN,2016-01,0.34
560754,노원구,2016-01-01,59.82,7,2005.0,2016,30800,NaN,2016-01,0.34
560755,서초구,2016-01-01,134.36,12,2005.0,2016,94000,NaN,2016-01,0.34
560756,양천구,2016-01-01,49.99,2,2000.0,2016,21000,NaN,2016-01,0.34


### 원유컬럼 합치기

In [90]:
oil = pd.read_csv('16-22유가.csv', encoding='euc-kr')
oil = oil.T.reset_index()

In [91]:
oil

,index,0,1,2
0,기준원유,원유,원유,원유
1,항목,국제유가(Dubai) (US$/배럴),도입단가 (US$/배럴),도입물량 (백만배럴)
2,2016.01,26.9,34.0,82.7
3,2016.02,28.9,29.0,96.7
4,2016.03,35.2,30.9,85.9
...,...,...,...,...
81,2022.08,96.6,108.8,96.9
82,2022.09,91.0,106.0,85.6
83,2022.10,91.2,101.3,79.5
84,2022.11,86.3,97.2,82.9


In [93]:
oil = oil.drop(0)

In [94]:
oil.columns = oil.iloc[0]

In [97]:
oil = oil.drop(1)

In [103]:
oil.reset_index(drop=True, inplace = True)

In [104]:
oil

1,항목,국제유가(Dubai) (US$/배럴),도입단가 (US$/배럴),도입물량 (백만배럴)
0,2016.01,26.9,34.0,82.7
1,2016.02,28.9,29.0,96.7
2,2016.03,35.2,30.9,85.9
3,2016.04,39.0,36.4,87.6
4,2016.05,44.3,39.7,92.7
...,...,...,...,...
79,2022.08,96.6,108.8,96.9
80,2022.09,91.0,106.0,85.6
81,2022.10,91.2,101.3,79.5
82,2022.11,86.3,97.2,82.9


In [105]:
oil = oil.rename(columns={'항목' : '계약일'})

In [108]:
oil.drop(columns = ['도입단가 (US$/배럴)', '도입물량 (백만배럴)'], inplace = True)

In [109]:
oil

1,계약일,국제유가(Dubai) (US$/배럴)
0,2016.01,26.9
1,2016.02,28.9
2,2016.03,35.2
3,2016.04,39.0
4,2016.05,44.3
...,...,...
79,2022.08,96.6
80,2022.09,91.0
81,2022.10,91.2
82,2022.11,86.3


In [110]:
oil['계약일'] = pd.to_datetime(oil['계약일'], format='%Y-%m-%d')

In [111]:
oil

1,계약일,국제유가(Dubai) (US$/배럴)
0,2016-01-01,26.9
1,2016-02-01,28.9
2,2016-03-01,35.2
3,2016-04-01,39.0
4,2016-05-01,44.3
...,...,...
79,2022-08-01,96.6
80,2022-09-01,91.0
81,2022-10-01,91.2
82,2022-11-01,86.3


In [113]:
oil['월'] = oil['계약일'].dt.to_period('M')

In [115]:
merged_df3 = pd.merge(merged_df2, oil, on='월', how='left')

In [116]:
merged_df3

,자치구명,계약일_x,건물면적(㎡),층,건축년도,연도,물건금액(만원),Close,월,FEDFUNDS,계약일,국제유가(Dubai) (US$/배럴)
0,동대문구,2022-12-31,84.97,11,2003.0,2022,76500,NaN,2022-12,4.10,2022-12-01,77.2
1,구로구,2022-12-31,59.88,4,2011.0,2022,85000,NaN,2022-12,4.10,2022-12-01,77.2
2,노원구,2022-12-31,41.30,12,1988.0,2022,39000,NaN,2022-12,4.10,2022-12-01,77.2
3,동대문구,2022-12-31,59.98,12,2011.0,2022,80000,NaN,2022-12,4.10,2022-12-01,77.2
4,송파구,2022-12-31,100.82,1,1988.0,2022,185000,NaN,2022-12,4.10,2022-12-01,77.2
...,...,...,...,...,...,...,...,...,...,...,...,...
560753,구로구,2016-01-01,84.85,18,1998.0,2016,30500,NaN,2016-01,0.34,2016-01-01,26.9
560754,노원구,2016-01-01,59.82,7,2005.0,2016,30800,NaN,2016-01,0.34,2016-01-01,26.9
560755,서초구,2016-01-01,134.36,12,2005.0,2016,94000,NaN,2016-01,0.34,2016-01-01,26.9
560756,양천구,2016-01-01,49.99,2,2000.0,2016,21000,NaN,2016-01,0.34,2016-01-01,26.9


In [118]:
# oil 계약일 삭제
merged_df3 = merged_df3.drop(columns = ['계약일'])

KeyError: "['계약일'] not found in axis"

In [119]:
merged_df3

,자치구명,계약일_x,건물면적(㎡),층,건축년도,연도,물건금액(만원),Close,월,FEDFUNDS,국제유가(Dubai) (US$/배럴)
0,동대문구,2022-12-31,84.97,11,2003.0,2022,76500,NaN,2022-12,4.10,77.2
1,구로구,2022-12-31,59.88,4,2011.0,2022,85000,NaN,2022-12,4.10,77.2
2,노원구,2022-12-31,41.30,12,1988.0,2022,39000,NaN,2022-12,4.10,77.2
3,동대문구,2022-12-31,59.98,12,2011.0,2022,80000,NaN,2022-12,4.10,77.2
4,송파구,2022-12-31,100.82,1,1988.0,2022,185000,NaN,2022-12,4.10,77.2
...,...,...,...,...,...,...,...,...,...,...,...
560753,구로구,2016-01-01,84.85,18,1998.0,2016,30500,NaN,2016-01,0.34,26.9
560754,노원구,2016-01-01,59.82,7,2005.0,2016,30800,NaN,2016-01,0.34,26.9
560755,서초구,2016-01-01,134.36,12,2005.0,2016,94000,NaN,2016-01,0.34,26.9
560756,양천구,2016-01-01,49.99,2,2000.0,2016,21000,NaN,2016-01,0.34,26.9


### 경기 부동산 거래량 컬럼 합치기

In [133]:
budong = pd.read_csv('경기 부동산 거래량.csv', encoding='utf-8')
budong = budong.T
budong = budong.reset_index()
budong

,index,0
0,지역,경기
1,통계명,아파트매매_월별
2,단위,동(호)수
3,'16.01,"9,894"
4,'16.02,"9,584"
...,...,...
82,'22.08,"3,657"
83,'22.09,"3,285"
84,'22.10,"3,270"
85,'22.11,"3,933"


In [155]:
budong.re

2,단위,동(호)수
3,'16.01,"9,894"
4,'16.02,"9,584"
5,'16.03,"12,628"
6,'16.04,"13,526"
7,'16.05,"15,668"
...,...,...
82,'22.08,"3,657"
83,'22.09,"3,285"
84,'22.10,"3,270"
85,'22.11,"3,933"


In [156]:
budong = budong.rename(columns={'단위': '계약일', '동(호)수': '부동산 거래량'})

In [158]:
budong.reset_index(drop=True,inplace=True)

In [159]:
budong

2,계약일,부동산 거래량
0,'16.01,"9,894"
1,'16.02,"9,584"
2,'16.03,"12,628"
3,'16.04,"13,526"
4,'16.05,"15,668"
...,...,...
79,'22.08,"3,657"
80,'22.09,"3,285"
81,'22.10,"3,270"
82,'22.11,"3,933"


In [162]:
budong['계약일'] = budong['계약일'].str.replace("'", "") 

In [169]:
budong['계약일'] = pd.to_datetime(budong['계약일'], format='%y.%m')

In [170]:
budong

2,계약일,부동산 거래량
0,2016-01-01,"9,894"
1,2016-02-01,"9,584"
2,2016-03-01,"12,628"
3,2016-04-01,"13,526"
4,2016-05-01,"15,668"
...,...,...
79,2022-08-01,"3,657"
80,2022-09-01,"3,285"
81,2022-10-01,"3,270"
82,2022-11-01,"3,933"


In [171]:
budong['월'] = budong['계약일'].dt.to_period('M')

In [172]:
merged_df4 = pd.merge(merged_df3, budong, on='월', how='left')

In [174]:
merged_df4 = merged_df4.drop(columns = ['계약일'])

In [177]:
merged_df4

,자치구명,계약일_x,건물면적(㎡),층,건축년도,연도,물건금액(만원),Close,월,FEDFUNDS,국제유가(Dubai) (US$/배럴),부동산 거래량
0,동대문구,2022-12-31,84.97,11,2003.0,2022,76500,NaN,2022-12,4.10,77.2,"3,698"
1,구로구,2022-12-31,59.88,4,2011.0,2022,85000,NaN,2022-12,4.10,77.2,"3,698"
2,노원구,2022-12-31,41.30,12,1988.0,2022,39000,NaN,2022-12,4.10,77.2,"3,698"
3,동대문구,2022-12-31,59.98,12,2011.0,2022,80000,NaN,2022-12,4.10,77.2,"3,698"
4,송파구,2022-12-31,100.82,1,1988.0,2022,185000,NaN,2022-12,4.10,77.2,"3,698"
...,...,...,...,...,...,...,...,...,...,...,...,...
560753,구로구,2016-01-01,84.85,18,1998.0,2016,30500,NaN,2016-01,0.34,26.9,"9,894"
560754,노원구,2016-01-01,59.82,7,2005.0,2016,30800,NaN,2016-01,0.34,26.9,"9,894"
560755,서초구,2016-01-01,134.36,12,2005.0,2016,94000,NaN,2016-01,0.34,26.9,"9,894"
560756,양천구,2016-01-01,49.99,2,2000.0,2016,21000,NaN,2016-01,0.34,26.9,"9,894"


### 한국금리 컬럼 합치기

In [182]:
kor = pd.read_csv('금리(16-22).csv', encoding='euc-kr')

In [183]:
kor

,날짜,국고채?3년(평균),국고채?5년(평균),국고채?10년(평균),회사채?3년(평균),CD?91물(평균),"콜금리(1일물,평균)",기준금리
0,Jan-16,1.63,1.77,2.03,2.10,1.67,1.49,1.50
1,Feb-16,1.47,1.59,1.83,1.95,1.64,1.49,1.50
2,Mar-16,1.50,1.61,1.85,1.98,1.63,1.49,1.50
3,Apr-16,1.47,1.57,1.81,1.94,1.61,1.49,1.50
4,May-16,1.46,1.55,1.79,1.91,1.58,1.49,1.50
...,...,...,...,...,...,...,...,...
79,Aug-22,3.25,3.30,3.32,4.23,2.79,2.26,2.50
80,Sep-22,3.90,3.94,3.90,4.90,3.01,2.53,2.50
81,Oct-22,4.24,4.29,4.27,5.44,3.69,2.96,3.00
82,Nov-22,3.90,3.95,3.92,5.49,3.99,3.10,3.25


In [184]:
kor['날짜'] = pd.to_datetime(kor['날짜'], format='%b-%y')

In [185]:
kor

,날짜,국고채?3년(평균),국고채?5년(평균),국고채?10년(평균),회사채?3년(평균),CD?91물(평균),"콜금리(1일물,평균)",기준금리
0,2016-01-01,1.63,1.77,2.03,2.10,1.67,1.49,1.50
1,2016-02-01,1.47,1.59,1.83,1.95,1.64,1.49,1.50
2,2016-03-01,1.50,1.61,1.85,1.98,1.63,1.49,1.50
3,2016-04-01,1.47,1.57,1.81,1.94,1.61,1.49,1.50
4,2016-05-01,1.46,1.55,1.79,1.91,1.58,1.49,1.50
...,...,...,...,...,...,...,...,...
79,2022-08-01,3.25,3.30,3.32,4.23,2.79,2.26,2.50
80,2022-09-01,3.90,3.94,3.90,4.90,3.01,2.53,2.50
81,2022-10-01,4.24,4.29,4.27,5.44,3.69,2.96,3.00
82,2022-11-01,3.90,3.95,3.92,5.49,3.99,3.10,3.25


In [187]:
kor = kor.rename(columns={'날짜': '계약일'})

In [188]:
kor['월'] = kor['계약일'].dt.to_period('M')

In [189]:
kor

,계약일,국고채?3년(평균),국고채?5년(평균),국고채?10년(평균),회사채?3년(평균),CD?91물(평균),"콜금리(1일물,평균)",기준금리,월
0,2016-01-01,1.63,1.77,2.03,2.10,1.67,1.49,1.50,2016-01
1,2016-02-01,1.47,1.59,1.83,1.95,1.64,1.49,1.50,2016-02
2,2016-03-01,1.50,1.61,1.85,1.98,1.63,1.49,1.50,2016-03
3,2016-04-01,1.47,1.57,1.81,1.94,1.61,1.49,1.50,2016-04
4,2016-05-01,1.46,1.55,1.79,1.91,1.58,1.49,1.50,2016-05
...,...,...,...,...,...,...,...,...,...
79,2022-08-01,3.25,3.30,3.32,4.23,2.79,2.26,2.50,2022-08
80,2022-09-01,3.90,3.94,3.90,4.90,3.01,2.53,2.50,2022-09
81,2022-10-01,4.24,4.29,4.27,5.44,3.69,2.96,3.00,2022-10
82,2022-11-01,3.90,3.95,3.92,5.49,3.99,3.10,3.25,2022-11


In [190]:
merged_df5 = pd.merge(merged_df4, kor, on='월', how='left')

In [191]:
merged_df5

,자치구명,계약일_x,건물면적(㎡),층,건축년도,연도,물건금액(만원),Close,월,FEDFUNDS,국제유가(Dubai) (US$/배럴),부동산 거래량,계약일,국고채?3년(평균),국고채?5년(평균),국고채?10년(평균),회사채?3년(평균),CD?91물(평균),"콜금리(1일물,평균)",기준금리
0,동대문구,2022-12-31,84.97,11,2003.0,2022,76500,NaN,2022-12,4.10,77.2,"3,698",2022-12-01,3.63,3.59,3.54,5.3,4.02,3.24,3.25
1,구로구,2022-12-31,59.88,4,2011.0,2022,85000,NaN,2022-12,4.10,77.2,"3,698",2022-12-01,3.63,3.59,3.54,5.3,4.02,3.24,3.25
2,노원구,2022-12-31,41.30,12,1988.0,2022,39000,NaN,2022-12,4.10,77.2,"3,698",2022-12-01,3.63,3.59,3.54,5.3,4.02,3.24,3.25
3,동대문구,2022-12-31,59.98,12,2011.0,2022,80000,NaN,2022-12,4.10,77.2,"3,698",2022-12-01,3.63,3.59,3.54,5.3,4.02,3.24,3.25
4,송파구,2022-12-31,100.82,1,1988.0,2022,185000,NaN,2022-12,4.10,77.2,"3,698",2022-12-01,3.63,3.59,3.54,5.3,4.02,3.24,3.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560753,구로구,2016-01-01,84.85,18,1998.0,2016,30500,NaN,2016-01,0.34,26.9,"9,894",2016-01-01,1.63,1.77,2.03,2.1,1.67,1.49,1.50
560754,노원구,2016-01-01,59.82,7,2005.0,2016,30800,NaN,2016-01,0.34,26.9,"9,894",2016-01-01,1.63,1.77,2.03,2.1,1.67,1.49,1.50
560755,서초구,2016-01-01,134.36,12,2005.0,2016,94000,NaN,2016-01,0.34,26.9,"9,894",2016-01-01,1.63,1.77,2.03,2.1,1.67,1.49,1.50
560756,양천구,2016-01-01,49.99,2,2000.0,2016,21000,NaN,2016-01,0.34,26.9,"9,894",2016-01-01,1.63,1.77,2.03,2.1,1.67,1.49,1.50


In [192]:
merged_df5 = merged_df5.drop(columns = ['계약일'])

In [193]:
merged_df5

,자치구명,계약일_x,건물면적(㎡),층,건축년도,연도,물건금액(만원),Close,월,FEDFUNDS,국제유가(Dubai) (US$/배럴),부동산 거래량,국고채?3년(평균),국고채?5년(평균),국고채?10년(평균),회사채?3년(평균),CD?91물(평균),"콜금리(1일물,평균)",기준금리
0,동대문구,2022-12-31,84.97,11,2003.0,2022,76500,NaN,2022-12,4.10,77.2,"3,698",3.63,3.59,3.54,5.3,4.02,3.24,3.25
1,구로구,2022-12-31,59.88,4,2011.0,2022,85000,NaN,2022-12,4.10,77.2,"3,698",3.63,3.59,3.54,5.3,4.02,3.24,3.25
2,노원구,2022-12-31,41.30,12,1988.0,2022,39000,NaN,2022-12,4.10,77.2,"3,698",3.63,3.59,3.54,5.3,4.02,3.24,3.25
3,동대문구,2022-12-31,59.98,12,2011.0,2022,80000,NaN,2022-12,4.10,77.2,"3,698",3.63,3.59,3.54,5.3,4.02,3.24,3.25
4,송파구,2022-12-31,100.82,1,1988.0,2022,185000,NaN,2022-12,4.10,77.2,"3,698",3.63,3.59,3.54,5.3,4.02,3.24,3.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560753,구로구,2016-01-01,84.85,18,1998.0,2016,30500,NaN,2016-01,0.34,26.9,"9,894",1.63,1.77,2.03,2.1,1.67,1.49,1.50
560754,노원구,2016-01-01,59.82,7,2005.0,2016,30800,NaN,2016-01,0.34,26.9,"9,894",1.63,1.77,2.03,2.1,1.67,1.49,1.50
560755,서초구,2016-01-01,134.36,12,2005.0,2016,94000,NaN,2016-01,0.34,26.9,"9,894",1.63,1.77,2.03,2.1,1.67,1.49,1.50
560756,양천구,2016-01-01,49.99,2,2000.0,2016,21000,NaN,2016-01,0.34,26.9,"9,894",1.63,1.77,2.03,2.1,1.67,1.49,1.50


### 가계부채 컬럼 합치기

In [199]:
gage = pd.read_csv('가계부채.csv')
gage = gage[['날짜', 'GDP대비가계부채비율']]
gage

,날짜,GDP대비가계부채비율
0,16-1,83.3
1,16-4,84.3
2,16-7,85.7
3,16-10,87.3
4,17-1,87.5
5,17-4,88.3
6,17-7,88.8
7,17-10,89.4
8,18-1,89.7
9,18-4,90.3


In [200]:
gage = gage.rename(columns={'날짜': '계약일'})

In [203]:
gage['계약일'] = pd.to_datetime(gage['계약일'], format='%y-%m')

In [204]:
gage

,계약일,GDP대비가계부채비율
0,2016-01-01,83.3
1,2016-04-01,84.3
2,2016-07-01,85.7
3,2016-10-01,87.3
4,2017-01-01,87.5
5,2017-04-01,88.3
6,2017-07-01,88.8
7,2017-10-01,89.4
8,2018-01-01,89.7
9,2018-04-01,90.3


In [205]:
gage['월'] = gage['계약일'].dt.to_period('M')

In [207]:
merged_df6 = pd.merge(merged_df5, gage, on='월', how='left')

In [209]:
merged_df6 = merged_df6.drop(columns = ['계약일'])

In [210]:
merged_df6

,자치구명,계약일_x,건물면적(㎡),층,건축년도,연도,물건금액(만원),Close,월,FEDFUNDS,국제유가(Dubai) (US$/배럴),부동산 거래량,국고채?3년(평균),국고채?5년(평균),국고채?10년(평균),회사채?3년(평균),CD?91물(평균),"콜금리(1일물,평균)",기준금리,GDP대비가계부채비율
0,동대문구,2022-12-31,84.97,11,2003.0,2022,76500,NaN,2022-12,4.10,77.2,"3,698",3.63,3.59,3.54,5.3,4.02,3.24,3.25,NaN
1,구로구,2022-12-31,59.88,4,2011.0,2022,85000,NaN,2022-12,4.10,77.2,"3,698",3.63,3.59,3.54,5.3,4.02,3.24,3.25,NaN
2,노원구,2022-12-31,41.30,12,1988.0,2022,39000,NaN,2022-12,4.10,77.2,"3,698",3.63,3.59,3.54,5.3,4.02,3.24,3.25,NaN
3,동대문구,2022-12-31,59.98,12,2011.0,2022,80000,NaN,2022-12,4.10,77.2,"3,698",3.63,3.59,3.54,5.3,4.02,3.24,3.25,NaN
4,송파구,2022-12-31,100.82,1,1988.0,2022,185000,NaN,2022-12,4.10,77.2,"3,698",3.63,3.59,3.54,5.3,4.02,3.24,3.25,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560753,구로구,2016-01-01,84.85,18,1998.0,2016,30500,NaN,2016-01,0.34,26.9,"9,894",1.63,1.77,2.03,2.1,1.67,1.49,1.50,83.3
560754,노원구,2016-01-01,59.82,7,2005.0,2016,30800,NaN,2016-01,0.34,26.9,"9,894",1.63,1.77,2.03,2.1,1.67,1.49,1.50,83.3
560755,서초구,2016-01-01,134.36,12,2005.0,2016,94000,NaN,2016-01,0.34,26.9,"9,894",1.63,1.77,2.03,2.1,1.67,1.49,1.50,83.3
560756,양천구,2016-01-01,49.99,2,2000.0,2016,21000,NaN,2016-01,0.34,26.9,"9,894",1.63,1.77,2.03,2.1,1.67,1.49,1.50,83.3


In [211]:
merged_df6.to_csv('df_1.csv', encoding='utf-8-sig')